#Imports and Installations

In [1]:
!pip install jupyter-dash

     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 1.0MB 16.5MB/s 
     |████████████████████████████████| 3.5MB 25.7MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 1.8MB 19.4MB/s 
     |████████████████████████████████| 358kB 31.3MB/s 
  Created wheel for dash: filename=dash-1.20.0-cp37-none-any.whl size=85845 sha256=e641ef5f4cc504d65ee108b70042fce50d02113b7e5b0071fcd591347a0cbfd8
  Stored in directory: /root/.cache/pip/wheels/e6/33/41/ce361d0d1da277d2350f815b10f2ab1e331592d93ef2b07e29
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-cp37-none-any.whl size=1014873 sha256=ccc8850bf6cf7828a7f1460ab0c53ec108e8214380ac339a71d1e046687003cb
  Stored in directory: /root/.cache/pip/wheels/03/a9/c5/dd5815c601b0ede164c223ffd7bafebde716ca57de06ef8aec
  Created wheel for dash-core-components: filename=dash_core_components-1.16.0-cp37-none-any.whl size=3540992 sha256=a6bc45cd03e61705074cd4

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
import doctest
import copy
from math import ceil
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data

In [3]:
df=[]

#UniCycle Model

In [4]:
#uncomment this decorator to test your code
def unicycle_model(curr_pose, v, w, dt=1.0):
    # refer to the kinematic equations of a unicycle model
    x_old, y_old, thetha_old = curr_pose[0], curr_pose[1], curr_pose[2]
    x = x_old + v * np.cos(thetha_old) *dt
    y = y_old + v * np.sin(thetha_old) *dt
    theta = thetha_old + w* dt
    
    # Keep theta bounded between [-pi, pi]
    theta = np.arctan2(np.sin(theta), np.cos(theta))
    return (x, y, theta)

In [5]:

def get_open_loop_commands_unicycle_imp(route, vc_fast, vc_slow, wc, dt =1):
    all_w = []
    all_v = []
    for (dir,amt) in route:
        if dir == "straight":
            t_straight = ceil((amt/vc_fast)/dt)
            all_w += [0]*t_straight
            all_v += [vc_fast] * t_straight
        else:
            amt = (amt/180) * np.pi
            t_turn = ceil((amt/wc)/dt)
            all_v += [vc_slow] * t_turn
            if dir == "right":
                all_w += [-wc]*t_turn
            elif dir == "left":
                all_w += [wc]*t_turn
    return all_v, all_w

In [6]:
def generate_path_unicycle(cmd = [("straight", 5), ("right", 90), ("straight", 6), ("left", 85)], vc_fast = 1, vc_slow = 0.5, wc=np.pi/12,dt=0.1,pose=np.array([0, 0, np.pi/2])):
    global df

    all_v, all_w = get_open_loop_commands_unicycle_imp(cmd,vc_fast,vc_slow,wc,dt)
    robot_trajectory = []
    robot_trajectory.append(list(pose))
    for v, w in zip(all_v, all_w):
        #instruction to take v, w and compute new pose    
        pose = unicycle_model(pose, v,w,dt) ################
        # store new pose
        robot_trajectory.append(list(pose))
    robot_trajectory = np.array(robot_trajectory)
    df_array = np.vstack((robot_trajectory[:,0],robot_trajectory[:,1]))
    df_array = np.transpose(df_array)
    df = pd.DataFrame(df_array, columns = ['x','y'])

#Bicycle Model

In [7]:
def bicycle_model(curr_pose, v, delta, dt=1.0):
    # refer to the kinematic equations of a bicycle model
    x_old, y_old, thetha_old = curr_pose[0], curr_pose[1], curr_pose[2]
    x = x_old + v * np.cos(thetha_old) *dt
    y = y_old + v * np.sin(thetha_old) *dt
    w = v * np.tan(delta)/0.9
    theta = thetha_old + w* dt
    
    theta = np.arctan2(np.sin(theta), np.cos(theta))
    return (x, y, theta)

In [8]:
def get_open_loop_commands_bicycle(route, vc_fast,vc_slow, deltac, dt=1):
    all_delta = []
    all_v = []
    L = 0.9
    wc = vc_slow * np.tan(deltac)/L
    for (dir,amt) in route:
        if dir == "straight":
            t_straight = ceil((amt/vc_fast)/dt)
            all_delta += [0]*t_straight
            all_v += [vc_fast] * t_straight
        else:
            amt = (amt/180) * np.pi
            t_turn = ceil((amt/wc)/dt)
            all_v += [vc_slow] * t_turn
            if dir == "right":
                all_delta += [-deltac]*t_turn
            elif dir == "left":
                all_delta += [deltac]*t_turn
    return all_v, all_delta

In [9]:
def generate_path_bicycle(cmd = [("straight", 5), ("right", 90), ("straight", 6), ("left", 85)], vc_fast = 1, vc_slow = 0.5, deltac=np.pi/12,dt=0.1,pose=np.array([0, 0, np.pi/2])):
    global df

    all_v, all_delta = get_open_loop_commands_bicycle(cmd,vc_fast,vc_slow,deltac,dt)
    robot_trajectory = []
    robot_trajectory.append(list(pose))
    for v, delta in zip(all_v, all_delta):
        #instruction to take v, w and compute new pose    
        pose = bicycle_model(pose, v,delta,dt) ################
        # store new pose
        robot_trajectory.append(list(pose))
    robot_trajectory = np.array(robot_trajectory)
    df_array = np.vstack((robot_trajectory[:,0],robot_trajectory[:,1]))
    df_array = np.transpose(df_array)
    df = pd.DataFrame(df_array, columns = ['x','y'])

#Dashboard

You now know the following

1. Generate open-loop control from a given route

2. Simulate vehicular robot motion using bicycle/ unicycle model

Imagine you want to make an utility for your co-workers to try and understand vehicle models. 
Dashboards are common way to do this.

There are several options out there : Streamlit, Voila, Observable etc

Follow this
<a href="https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e">Medium post</a> on Jupyter Dash and see how to package what you learnt today in an interactive manner

Here is a   <a href="https://stackoverflow.com/questions/53622518/launch-a-dash-app-in-a-google-colab-notebook">stackoverflow question </a> on how to run dash applications on Collab

What can you assume?
+ Fix $v,\omega$ or $v,\delta$ depending on the model (users can still pick the actual value)
+ fixed wheelbase for bicycle model

Users can choose 
+ unicycle and bicycle models
+ A pre-configured route ("S", "inverted-S", "figure-of-eight" etc)
+ 1 of 3 values for $v, \omega$ (or $\delta$) 

In [10]:
pi = round(np.pi,3)
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Kinematic Modelling"),
    dcc.Graph(id='graph'),
    html.Label([
        "Kinematic Model",
        dcc.Dropdown(
            id='model-dropdown', clearable=False,
            value='unicycle', options=[
                {'label': c, 'value': c}
                for c in {'unicycle', 'bicycle'}
            ])
    ]),
    html.Div(),
    html.Label([
        "Pre Configured Routes",
        dcc.Dropdown(
            id='route-dropdown', clearable=False,
            value='S', options=[
                {'label': c, 'value': c}
                for c in {'S', 'inverted-S', 'figure-of-eight'}
            ])
    ]),
    html.Div(),
    html.Label([
        "Fast Velocity - used in straight segments(m/s)\t",
        dcc.Input(
            id="vc_fast",
            type="text",
            placeholder="Enter value for fast velocity",value = 1
        )
    ]),
    html.Div(),
    html.Label([
        "Slow Velocity - used in turning segments(m/s)\t",
        dcc.Input(
            id="vc_slow",
            type="text",
            placeholder="Enter value for slow velocity",value = 0.5
        )
    ]),
    html.Div(),
    html.Label([
        "Omega - Angular Velocity (rad/s) = \t pi/\t",
        dcc.Input(
            id="wc",
            type="text",
            placeholder="Enter value for n where omega = pi/n", value = 4
        )
    ]),
    html.Div(),
    html.Label([
        "Delta_C (rad) = \t pi/\t",
        dcc.Input(
            id="delta_c",
            type="text",
            placeholder="Enter value for n where delta_c = pi/n", value = 6
        )
    ]),
    html.Div(),
    html.Label([
        "Sampling Time (sec)\t",
        dcc.Input(
            id="dt",
            type="text",
            placeholder="Enter resolution value for sampling time", value = 0.1
        )
    ]),
    html.Div(),
    html.Label([
        "Wheelbase(m)\t",
        dcc.Input(
            id="L",
            type="text",
            placeholder="Enter value for wheelbase", value = 0.9,disabled = True
        )
    ]),
    
])# Define callback to update graph
@app.callback(
   Output('delta_c','disabled'), Output('wc','disabled'),
   [Input(component_id='model-dropdown', component_property='value')])

def show_hide_element(visibility_state):
    if visibility_state == 'unicycle':
        return True, False
    elif visibility_state == 'bicycle':
        return False, True


@app.callback(
    Output('graph', 'figure'),
    Input(component_id='model-dropdown', component_property='value'),
    Input(component_id='route-dropdown', component_property='value'),
    Input(component_id='vc_fast', component_property='value'),
    Input(component_id='vc_slow', component_property='value'),
    Input(component_id='dt', component_property='value'),
    Input(component_id='wc', component_property='value'),
    Input(component_id='delta_c', component_property='value')
)
# def update_figure(vc_fast,vc_slow,dt,w):
def update_figure(model,route,vc_fast,vc_slow,dt,wc,delta_c):
    global df

    vc_fast = float(vc_fast)
    dt = float(dt)
    vc_slow =float(vc_slow)
    wc = np.pi / float(wc)
    delta_c = np.pi / float(delta_c)

    if route=='S':
        cmd = [("left", 30), ("left", 190),  ("straight", 1),("right", 190), ("right", 30)]
    elif route=='inverted-S':
        cmd = [("right", 30), ("right", 190),  ("straight", 1),("left", 190), ("left", 30)]
    elif route=='figure-of-eight':
        cmd = [("right", 30), ("right", 192),  ("straight", 1.1),("left", 266),("straight", 1.8)]
    
    if model =='unicycle':
        generate_path_unicycle(cmd = cmd, vc_fast =vc_fast, dt =dt, vc_slow=vc_slow, wc=wc)
    else:
        generate_path_bicycle(cmd = cmd, vc_fast =vc_fast, dt =dt, vc_slow=vc_slow, deltac=delta_c)
    plt = px.line(
        df, x="x", y="y", 
        title="Path"
    )
    plt.layout.yaxis.scaleanchor = 'x' 
    # plt.axes().set_aspect("equal","datalim")

    # return px.line(
    #     df, x="x", y="y", 
    #     title="Path"
    # )# Run app and display result inline in the notebook
    return plt

app.run_server(mode='external',debug=True)

Dash app running on:


<IPython.core.display.Javascript object>